# Using Machine Learning to Help Small Businesses
If there is one thing businesses are learning, it's that proper data analytics isn't enough to gain a competetive edge, but has become a necessary ticket to compete. Sadly, many small businesses have neither the resources nor the know-how to take advantage of what data is available to them. With this project, the aim is to build a machine learning algorithm that can help any small business get a better sense of how they are presenting themselves to customers, who their direct competitors are, and how they compare to those competitors.

## Yelp
This primary source of data for this analysis is Yelp. We will use the Yelp API in conjunction with web scraping techniques and NLP sentiment analysis to gather and analyze customer feedback.

In [2]:
import pandas as pd
import numpy as np
import json
from yelpapi import YelpAPI
from sklearn.cluster import KMeans

# Load Yelp API token data
f = open('access_tokens.json', 'r')
tokens = json.load(f)
section_tokens = tokens['yelp']['first_yelp_app_rwilleyii']
client_id = section_tokens['Client ID']
api_key = section_tokens['API Key']

# Load Yelp API
yelp_api = YelpAPI(api_key)

## Business Selection Criteria
This model relies on 3 inputs from the user to generate results:

1. **Company Name**: This must be input exactly how the name appears on Yelp. It is advised to use copy/paste directly from Yelp.
2. **Search Term**: Consider a customer that doesn't know the business exists, but could benefit from the service/product the company offers. What search terms do you expect your generic customer to search for? (I.e "mexican food", "rooftop bar", "office supplies", etc.)
3. **Location**: Enter a location in the way you expect most customers to search. In most cases, this will be simply City, ST, but it could be zip cod as well.

In [3]:
# Get user preferences
company_name = input('Please enter company name as it appears on Yelp.com: ')
term = input('For someone who has never been to your restaurant, what terms might they search on Yelp to find you? ')
location = input('Where is your restaurant located? (city ST, ZIP, etc.) ')

Please enter company name as it appears on Yelp.com:  Chano's Cantina
For someone who has never been to your restaurant, what terms might they search on Yelp to find you?  mexican restaurant bar
Where is your restaurant located? (city ST, ZIP, etc.)  astoria ny


In [4]:
company_details = pd.DataFrame(data=[[company_name, term, location]], 
                               columns=['company_name', 'search_terms', 'location'])
company_details.to_csv('firm_data/inputs.csv')

## Generate Results from Yelp API
Using the variables above, return the top 50 results. The results should include the business in question; if not, there are likely more fundamental issues that need to be addressed before this model will be helpful. For example, if you are an italian restaurant but don't show up in search results for italian restaurants in the same neighborhood, the company is probably improperly listed on Yelp. The company should check to make sure the proper tags are in place or contact Yelp to find out what is going on.

In [7]:
search_limit = 50

response = yelp_api.search_query(term = term,
                                 location = location,
                                 limit = search_limit)

type(response)

dict

### Save Search Results to Dataframe

In [6]:
cols = list(response['businesses'][0].keys())
data = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    data = data.append(biz, ignore_index=True)

data = data[data['is_closed']==False]
data.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,EHUNrnIgnhwTnpOm3gEESg,tacuba-astoria,Tacuba,https://s3-media4.fl.yelpcdn.com/bphoto/Q6jPz4...,False,https://www.yelp.com/biz/tacuba-astoria?adjust...,645,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 40.75585, 'longitude': -73.92447}","[delivery, pickup]",$$,"{'address1': '35-01 36th St', 'address2': '', ...",+17187862727,(718) 786-2727,1229.351705
1,yvva7IYpD6J7OfKlCdQrkw,mi-espiguita-taqueria-astoria,Mi Espiguita Taqueria,https://s3-media2.fl.yelpcdn.com/bphoto/TEho39...,False,https://www.yelp.com/biz/mi-espiguita-taqueria...,113,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 40.7612033639422, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '32-44 31st St', 'address2': '', ...",+17187775648,(718) 777-5648,714.301080
2,anBWvFPOce4nl3qLX-H6Rg,el-mero-mero-long-island-city,El Mero Mero,https://s3-media2.fl.yelpcdn.com/bphoto/7tp30k...,False,https://www.yelp.com/biz/el-mero-mero-long-isl...,81,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 40.753834323302, 'longitude': -73...","[delivery, pickup]",$$,"{'address1': '35-57 38th St', 'address2': '', ...",+17183922444,(718) 392-2444,1457.114073
3,2jjxdN9FXWVdx6DEX83krQ,hoja-santa-astoria,Hoja Santa,https://s3-media3.fl.yelpcdn.com/bphoto/xqo13E...,False,https://www.yelp.com/biz/hoja-santa-astoria?ad...,33,"[{'alias': 'newmexican', 'title': 'New Mexican...",4.5,"{'latitude': 40.7638197, 'longitude': -73.914523}",[],NaN,"{'address1': '40-17 30th Ave', 'address2': '',...",,,751.531469
4,xrKa2SpgWzPEEbji3Iw2DQ,the-calaveras-new-york,The Calaveras,https://s3-media3.fl.yelpcdn.com/bphoto/u1T_yd...,False,https://www.yelp.com/biz/the-calaveras-new-yor...,103,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 40.79956, 'longitude': -73.96241}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '949 Columbus Ave', 'address2': N...",+16464846533,(646) 484-6533,4963.939653


In [11]:
data.head().to_csv('data_head.csv')

In [ ]:
data.to_csv(f'firm_data/{company_name}_search_results.csv')